In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline

In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none func

In [2]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as message
from biogeme import expressions as ex
import pandas as pd
import numpy as np
import xlsxwriter

# Calibration
## Estimation of the model parameters
quetzal_germany is being estimated using [PandasBiogeme](https://biogeme.epfl.ch/). This notebook estimates calibration parameters for the model's utility functions.
- Documentation and reference: [Bierlaire, M. (2020). A short introduction to PandasBiogeme. Technical report TRANSP-OR 200605. Transport and Mobility Laboratory, ENAC, EPFL.](https://transp-or.epfl.ch/documents/technicalReports/Bier20.pdf)
- Tutorial: https://www.youtube.com/watch?v=OiM94B8WayA

### Model formulation
The model consists of systematic utility functions, one for each mode. They comprise an alternaive-specific constant (ASC), a distance-dependent part with travel time and cost summarised as generalised cost (GC), and a cost damping function F

$V_i = ASC + F(GC(T, C), b_{gc_i})$

Index i marks the demand group. I = {'commuting' (1), 'education' (2), 'shopping/medical' (3), 'business' (4), 'private' (6)}

Note: The cost variable already includes subscriptions

In [3]:
input_path = '../input/'
model_path = '../model/'

### Prepare the database

In [4]:
df = pd.read_csv(input_path + 'transport_demand/calibration_inter-cellular_trips_MiD2017.csv')
df = df[['cost_rail_short', 'cost_rail_long', 'cost_car', 'cost_coach', 'cost_bus', 'cost_walk', 'cost_air',
         'time_rail_short', 'time_rail_long', 'time_car', 'time_coach', 'time_bus', 'time_walk', 'time_air',
         'cost_rail', 'cost_road', 'time_rail', 'time_road',
         'mode_model', 'purpose_vp', 'car_avail', 'length']]
df.columns = ['C_RAIL_S', 'C_RAIL_L', 'C_CAR', 'C_COACH', 'C_BUS', 'C_NON_MOTOR', 'C_AIR',
              'T_RAIL_S', 'T_RAIL_L', 'T_CAR', 'T_COACH', 'T_BUS', 'T_NON_MOTOR', 'T_AIR',
              'C_RAIL', 'C_ROAD', 'T_RAIL', 'T_ROAD',
              'MODE', 'PURPOSE', 'CAR_AV', 'DIST']

In [5]:
inf = 1000
df = df.replace({np.inf:inf})

In [6]:
df.describe()

,C_RAIL_S,C_RAIL_L,C_CAR,C_COACH,C_BUS,C_NON_MOTOR,C_AIR,T_RAIL_S,T_RAIL_L,T_CAR,...,T_NON_MOTOR,T_AIR,C_RAIL,C_ROAD,T_RAIL,T_ROAD,MODE,PURPOSE,CAR_AV,DIST
count,94917.000000,94917.000000,94917.000000,94917.000000,94917.000000,94917.0,94917.000000,94917.000000,94917.000000,94917.000000,...,94917.000000,94917.000000,94917.000000,94917.000000,94917.000000,94917.000000,94917.000000,94917.000000,94917.000000,94917.000000
mean,1.301276,86.947799,1.527609,7.572003,3.250944,0.0,961.461744,93.544766,597.050884,80.318733,...,451.098820,972.293098,8.092171,4.383917,82.574130,197.630102,5.560268,3.654762,0.965201,95.483902
std,0.467537,56.149364,1.840845,6.731983,1.270851,0.0,179.546027,118.721222,435.093451,85.017232,...,449.687307,130.264335,15.777624,3.753519,88.911093,308.637355,1.350390,2.059627,0.187364,135.479350
min,0.000000,19.000000,0.066770,5.000000,0.000000,0.0,44.162224,12.433333,16.033333,10.400000,...,2.660412,3.974444,0.000000,0.000000,12.433333,13.200000,1.000000,1.000000,0.000000,0.265000
25%,1.470385,21.128156,0.400929,5.000000,2.740000,0.0,1000.000000,29.683333,119.883333,36.483333,...,75.916667,1000.000000,1.471910,3.600000,29.666667,40.533333,6.000000,1.000000,1.000000,29.561000
50%,1.476474,139.000000,0.603615,5.000000,4.000000,0.0,1000.000000,44.366667,1000.000000,49.850000,...,119.216667,1000.000000,1.478143,4.000000,44.033333,68.450000,6.000000,3.000000,1.000000,45.078000
75%,1.482707,139.000000,2.003906,5.000000,4.000000,0.0,1000.000000,91.750000,1000.000000,75.316667,...,1000.000000,1000.000000,1.500041,4.000000,89.233333,148.008333,6.000000,6.000000,1.000000,80.058000
max,1.763928,139.000000,11.478678,60.000000,4.000000,0.0,1000.000000,1000.000000,1000.000000,682.283333,...,1000.000000,1000.000000,139.000000,48.821179,681.516667,1000.000000,7.000000,6.000000,9.000000,1270.076000


In [7]:
# Scale time to hours
df['T_RAIL_Sc'] = df['T_RAIL'] / 60
df['T_RAIL_S_S'] = df['T_RAIL_S'] / 60
df['T_RAIL_L_S'] = df['T_RAIL_L'] / 60
df['T_CAR_S'] = df['T_CAR'] / 60
df['T_COACH_S'] = df['T_COACH'] / 60
df['T_BUS_S'] = df['T_BUS'] / 60
df['T_ROAD_S'] = df['T_ROAD'] / 60
df['T_AIR_S'] = df['T_AIR'] / 60
df['T_NON_MOTOR_S'] = df['T_NON_MOTOR'] / 60

In [8]:
# Make car availability binary
df['CAR_AV'] = df['CAR_AV'].replace({9:0})

In [9]:
# Add PT availabilities
df['RAIL_AV'] = (df['T_RAIL']!=inf).astype(int)
df['RAIL_S_AV'] = (df['T_RAIL_S']!=inf).astype(int)
df['RAIL_L_AV'] = (df['T_RAIL_L']!=inf).astype(int)
df['COACH_AV'] = (df['T_COACH']!=inf).astype(int)
df['BUS_AV'] = (df['T_BUS']!=inf).astype(int)
df['ROAD_AV'] = (df['T_ROAD']!=inf).astype(int)
df['AIR_AV'] = (df['T_AIR']!=inf).astype(int)
df['NON_MOTOR_AV'] = (df['C_NON_MOTOR']!=inf).astype(int)

In [10]:
# Merge rail and road PT
df['MODE'] = df['MODE'].replace({2:1, 3:4})
len(df.loc[df['MODE'].isin([2,3])])

0

In [11]:
# Remove trips where mode is car but the car availability is zero
# because it irritates the MLE algorithm
mask = ((df['MODE']==6) & (df['CAR_AV']==0))
drop_ratio = len(df.loc[mask])/len(df.loc[df['MODE']==6])
lengths = list(df.loc[mask, 'DIST'])
print('Share of car trips dropped: {}'.format(drop_ratio))
df = df.loc[~mask]
# Drop other modes equally
for m in [1,4]:
    n_drops = int(np.round(drop_ratio * len(df.loc[df['MODE']==m])))
    df = df.drop(df.loc[(df['MODE']==m) & (df['DIST']<max(lengths)) &
                        (df['DIST']>min(lengths))].sample(n_drops).index)
    print('Dropped ' + str(n_drops) + ' trips of mode ' + str(m))
print(len(df))

Share of car trips dropped: 0.014148505211025496
Dropped 106 trips of mode 1
Dropped 45 trips of mode 4
93589


In [12]:
# Check that PT trips have no infinity cost
for mode, col in zip([1, 4, 5], ['RAIL_AV', 'ROAD_AV', 'AIR_AV']):
    assert len(df.loc[((df['MODE']==mode) & ~(df[col]))].index) == 0, str(mode)

### Model variables
All columns are variables. DefineVariable creates a new column in the database.

#### Cost damping

Many modelling studies have shown that cost damping is required in order to flatten the tail of time/cost elasticities, i.e. decrease the impact of long distances on choice results to prevent from overestimation of time/cost parameters. Cost damping represents the property of decreasing marginal utility. It is commonly approached with Box-Cox transformations of generalised cost (usually defined as the sum of travel time and travel expenditures divided by the value of time). Daly (2010) proposes a hybrid function as a sum of the linear term and the narural logarithm of the same. However, the linear term still dominates cost on long distances. Rich (2015), main developer of the Danish National Transport Model, proposes a more complex spline function which successfully manages cost damping and even outperforms the Box-Cox transformation in terms of stability of elasticities.

In [13]:
# Using the hybrid linear-logarithmic version requires a new column
# for the logarithm of time in minutes
'''df['T_RAIL_S'] = np.log(df['T_RAIL'])
df['T_CAR_S'] = np.log(df['T_CAR'])
df['T_COACH_S'] = np.log(df['T_COACH'])
df['T_BUS_S'] = np.log(df['T_BUS'])
df['T_AIR_S'] = np.log(df['T_AIR'])
df['T_NON_MOTOR_S'] = np.log(df['T_NON_MOTOR'])'''
print('This functional form performes poorly')

This functional form performes poorly


#### Generalised cost

Both, travel time and monetary cost should be affected by cost damping measures. It is logical to define a generalised cost term `GC` with dimension of time units. This requires definition or estimation of values of time, in order to rescale monetary units, for all segments. Usually, the value of time (VoT) is distance-dependent. In the case of Germany, VoT can be taken from research undertaken within the Federal Government's transport study "Bundesverkehrswegeplan 2030": Axhausen et al. 2015. Ermittlung von Bewertungsansätzen für Reisezeiten und Zuverlässigkeit auf der Basis eines Modells für modale Verlagerungen im nicht-gewerblichen und gewerblichen Personenverkehr für die Bundesverkehrswegeplanung

In [13]:
# VoT from literature, distance-dependent, see cal19
VoT = pd.read_csv(input_path + 'vot.csv', header=[0,1], index_col=0)
VoT.sample(2)

,root,Fz1,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,...,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,Fz4,Fz6
,all,all,all,all,all,all,PT,PT,PT,PT,...,air,air,air,air,car,car,car,car,car,car
477,17.4830,16.7752,9.18,11.9,20.8884,13.4402,13.2572,10.3630,13.48,14.7930,...,46.2852,46.2852,73.3834,27.0912,17.3468,16.5290,10.5692,22.144,18.7864,13.4402
227,14.5876,14.0460,9.18,11.9,16.1312,11.5248,11.1672,8.9584,13.48,13.3984,...,22.1068,22.1068,25.9712,16.7996,14.5468,13.9336,9.0992,18.784,15.5792,11.5248


In [14]:
# Make distance integer
df['DIST'] = df['DIST'].apply(int)

In [15]:
df = df.loc[df['DIST']<=1000]
len(df)

93585

In [16]:
# Generate generalised cost
VoT = VoT.to_dict()
df['GC_RAIL'] = df['T_RAIL_Sc'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                   for c,d,p in zip(df['C_RAIL'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_S'] = df['T_RAIL_S_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_S'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_L'] = df['T_RAIL_L_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_L'], df['DIST'], df['PURPOSE'])]
df['GC_COACH'] = df['T_COACH_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                    for c,d,p in zip(df['C_COACH'], df['DIST'], df['PURPOSE'])]
df['GC_BUS'] = df['T_BUS_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                for c,d,p in zip(df['C_BUS'], df['DIST'], df['PURPOSE'])]
df['GC_ROAD'] = df['T_ROAD_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                  for c,d,p in zip(df['C_ROAD'], df['DIST'], df['PURPOSE'])]
df['GC_AIR'] = df['T_AIR_S'] + [c / VoT[('Fz'+str(p), 'air')][d]
                                for c,d,p in zip(df['C_AIR'], df['DIST'], df['PURPOSE'])]
df['GC_CAR'] = df['T_CAR_S'] + [c / VoT[('Fz'+str(p), 'car')][d]
                                for c,d,p in zip(df['C_CAR'], df['DIST'], df['PURPOSE'])]
df['GC_NON_MOTOR'] = df['T_NON_MOTOR_S']

In [17]:
# Create the initial database and make columns global variables
database = db.Database('MiD2017', df.copy())
globals().update(database.variables)
database.getSampleSize()

93585

### Estimation parameters

In [18]:
asc_rail = ex.Beta('asc_rail', 0, None, None, 1)
asc_rail_s = ex.Beta('asc_rail_s', 0, None, None, 0)
asc_rail_l = ex.Beta('asc_rail_l', 0, None, None, 0)
asc_coach = ex.Beta('asc_coach', 0, None, None, 0)
asc_bus = ex.Beta('asc_bus', 0, None, None, 0)
asc_road = ex.Beta('asc_road', 0, None, None, 0)
asc_air = ex.Beta('asc_air', 0, None, None, 0)
asc_car = ex.Beta('asc_car', 0, None, None, 0)
asc_non_motor = ex.Beta('asc_non_motor', 0, None, None, 0)

In [19]:
b_t = ex.Beta('b_t', 0, None, None, 0)
b_c = ex.Beta('b_c', 0, None, None, 0)
b_ac = ex.Beta('b_ac', 0, None, None, 0)

In [20]:
# non-linear time component
b_t2 = ex.Beta('b_t2', 0, None, None, 0)

In [21]:
# generalised cost function parameter
b_gc = ex.Beta('b_gc', 0, None, None, 0)

In [22]:
# Parameters for the nested logit structure
mu_pt = ex.Beta('mu_pt', 1, 1, 10, 0)

### Utility functions

In [23]:
'''# Aggregated formulation
V_RAIL = asc_rail + b_t * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR_S

# Aggregated formulation with non-linear perception of travel time
V_RAIL = asc_rail + b_t * T_RAIL + b_t2 * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH + b_t2 * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS + b_t2 * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR + b_t2 * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR + b_t2 * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR + b_t2 * T_NON_MOTOR_S'''
print('Deprected')

Deprected


In [23]:
# The cost damping function
def spline(x, beta=b_gc, Q=3, c0=0, c1=20, c2=40, c3=np.inf):
    alpha = [0, -beta/2*ex.Power(ex.log(c1),3),
             -beta/2*ex.log(c1)*(3*ex.Power(ex.log(c2),2)+ex.Power(ex.log(c1),2))] # for Q=3
    theta = [1, 3/2*ex.log(c1), 3*ex.log(c1)*ex.log(c2)] # for Q=3
    def component(x, q):
        return beta*theta[q-1]*ex.Power(ex.log(x),Q-q+1) + alpha[q-1]
    if x < c1:
        return component(x,1)
    elif x < c2:
        return component(x,2)
    elif x >= c2:
        return component(x,3)

In [24]:
# Aggregated formulation with damped generalised cost
V_RAIL = asc_rail + spline(GC_RAIL)
V_RAIL_S = asc_rail_s + spline(GC_RAIL_S)# + b_ac * AC_RAIL
V_RAIL_L = asc_rail_l + spline(GC_RAIL_L)# + b_ac * AC_RAIL
V_COACH = asc_coach + spline(GC_COACH)# + b_ac * AC_COACH
V_BUS = asc_bus + spline(GC_BUS)# + b_ac * AC_BUS
V_ROAD = asc_road + spline(GC_ROAD)
V_AIR = asc_air + spline(GC_AIR)# + b_ac * AC_AIR
V_CAR = asc_car + spline(GC_CAR)
V_NON_MOTOR = asc_non_motor + spline(GC_NON_MOTOR)

### Run the estimation

In [25]:
# Define level of verbosity
logger = message.bioMessage()
#logger.setSilent()
logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

In [26]:
# Map modes to utility functions
V = {1:V_RAIL,
#     1:V_RAIL_S,
#     2:V_RAIL_L,
#     3:V_COACH,
     4:V_ROAD,#BUS,
     5:V_AIR,
     6:V_CAR,
     7:V_NON_MOTOR}

In [27]:
# Map the availability of alternatives with MODE as key
# PT is always available
av = {1:RAIL_AV,#RAIL_S_AV,
#      2:RAIL_L_AV,
#      3:COACH_AV,
      4:ROAD_AV,#BUS_AV,
      5:AIR_AV,
      6:CAR_AV,
      7:NON_MOTOR_AV}

In [28]:
# Mode nests as tuples with nest name and dictionary where
# alternative IDs are mapped to alpha values. Missing ID's alpha is zero
# Alternatively use lists with mode ID without alpha
pt = mu_pt, [1, 4]#[1, 2, 3, 4]
air = 1, [5]
car = 1, [6]
walk = 1, [7]
nests = pt, air, car, walk

In [29]:
# Write results to an Excel file
writer = pd.ExcelWriter(input_path + 'estimation_results_car_own.xls', engine='xlsxwriter')

In [30]:
# Choose the multinomial logit model
mnl = models.loglogit(V, av, MODE)

In [31]:
model_mnl = bio.BIOGEME(database, mnl)
model_mnl.modelName = 'MNL'

In [32]:
results_mnl = model_mnl.estimate()

In [33]:
results = results_mnl.getEstimatedParameters()
for key, val in results_mnl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-1.659536,0.167532,-9.90579,0,0.184229,-9.00799,0
asc_car,2.443709,0.0132272,184.748,0,0.013379,182.653,0
asc_non_motor,-1.703558,0.0357237,-47.6871,0,0.034709,-49.0812,0
asc_road,-0.752482,0.0215393,-34.9354,0,0.0216001,-34.837,0
b_gc,-0.284448,0.00699102,-40.6875,0,0.015189,-18.7273,0
Number of estimated parameters,5.000000,,,,,,
Sample size,93585.000000,,,,,,
Excluded observations,0.000000,,,,,,
Init log likelihood,-126917.928747,.7g,,,,,
Final log likelihood,-38423.303704,.7g,,,,,


In [34]:
results.to_excel(writer, sheet_name=model_mnl.modelName)

In [35]:
# Choose the logarithmic nested logit model
nl = models.lognested(V, av, nests, MODE)

In [36]:
# Nested Logit
model_nl = bio.BIOGEME(database, nl)
model_nl.modelName = 'NL'

In [37]:
results_nl = model_nl.estimate()

In [38]:
results = results_nl.getEstimatedParameters()
for key, val in results_nl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-1.661887,0,0.16888,-9.84066,0,0.195819,-8.48685,0
asc_car,2.443644,0,0.0259605,94.1292,0,0.0721291,33.8788,0
asc_non_motor,-1.704027,0,0.039705,-42.9172,0,0.0713121,-23.8954,0
asc_road,-0.752449,0,0.0545059,-13.8049,0,0.157843,-4.76707,1.8692e-06
b_gc,-0.284485,0,0.010475,-27.1586,0,0.0209099,-13.6053,0
mu_pt,1.000000,1,0.0689491,14.5034,0,0.21311,4.6924,2.70016e-06
Number of estimated parameters,6.000000,,,,,,,
Sample size,93585.000000,,,,,,,
Excluded observations,0.000000,,,,,,,
Init log likelihood,-126917.928747,.7g,,,,,,


In [39]:
results.to_excel(writer, sheet_name=model_nl.modelName)

In [41]:
# Run the estimation by purpose
results = []
for p in [1,2,3,4,6]:
    for car_ownership in [0,1]:
        database = db.Database('MiD2017', df.copy())
        database.remove(PURPOSE!=p)
        database.remove(CAR_AV!=car_ownership)
        print('Sample size for purpose {}, car_ownership {}: {}'.format(
            p, car_ownership, database.getSampleSize()))
        model = bio.BIOGEME(database, mnl) # Choose the model formulation
        model.modelName = 'MNL_Fz' + str(p) + '_' + str(car_ownership) # Name it
        results.append(model.estimate()) # Estimation
        output = results[-1].getEstimatedParameters()
        # Add results to the Excel file
        for key, val in results[-1].getGeneralStatistics().items():
            output.loc[key] = [val[0], val[1]] + ['' for i in range(len(output.columns)-2)]
        output.to_excel(writer, sheet_name=model.modelName)

Sample size for purpose 1, car_ownership 0: 635
Sample size for purpose 1, car_ownership 1: 24608
Sample size for purpose 2, car_ownership 0: 115
Sample size for purpose 2, car_ownership 1: 3443
Sample size for purpose 3, car_ownership 0: 411
Sample size for purpose 3, car_ownership 1: 22571
Sample size for purpose 4, car_ownership 0: 99
Sample size for purpose 4, car_ownership 1: 5504
Sample size for purpose 6, car_ownership 0: 853
Sample size for purpose 6, car_ownership 1: 35346


In [42]:
writer.save()

In [43]:
# Generate LaTeX code
file = open(input_path + 'estimation_results_LaTeX_code.txt', 'w')
for r in results:
    file.write(r.getLaTeX())
file.close()